In [1]:
#Importar librerías

import pandas as pd
import matplotlib.pyplot as plt
import os

#Crear dataframe a partir de un CSV(comma separate values)
dataframe = pd.read_csv("csv/Tasa_de_desocupación_INE.csv")

#Obtener columnas necesarias para graficar o sacar conclusiones
dataframe_clean = dataframe[["DTI_CL_TRIMESTRE_MOVIL","DTI_CL_REGION","Región","Sexo","Value"]]

In [2]:
#Functions

def prepare_data(df, year):
    
    """
    Filtra y prepara los datos para un año específico.
    
    Args:
    - df (DataFrame): DataFrame original con los datos.
    - year (str): Año a filtrar en formato 'YYYY'.
    
    Returns:
    - DataFrame: DataFrame filtrado y preparado por mes y sexo.
    """
    
    df_filtered = df.copy()
    
    # Filtrar por año y extraer mes
    df_filtered = df_filtered[df_filtered['DTI_CL_TRIMESTRE_MOVIL'].str.startswith(year)]
    df_filtered.loc[:, 'Mes'] = df_filtered['DTI_CL_TRIMESTRE_MOVIL'].str[-2:].astype(int)
    
    # Mapear número de mes a nombre de mes
    months_dict = {
        1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril',
        5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto',
        9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'
    }
    df_filtered = df_filtered[df_filtered['Mes'].isin(months_dict.keys())]  # Filtrar solo los meses presentes
    df_filtered['Mes'] = df_filtered['Mes'].map(months_dict)  # Mapear el nombre del mes
    
    # Pivotar para tener sexo como columnas y mes como índice
    df_pivot = df_filtered.pivot_table(index=['Región', 'Mes'], columns='Sexo', values='Value').reset_index()
    
    # Ordenar por región y luego por mes cronológicamente
    df_pivot['Mes'] = pd.Categorical(df_pivot['Mes'], categories=months_dict.values(), ordered=True)
    df_pivot = df_pivot.sort_values(by=['Región', 'Mes'])
    
    return df_pivot

def plot_by_region(df, year):
    """
    Grafica las tasas de desocupación por región para un año específico.
    
    Args:
    - df (DataFrame): DataFrame preparado con los datos por mes y sexo.
    - year (str): Año a mostrar en los gráficos.
    """
    # Crear la carpeta para guardar los gráficos si no existe
    folder_name = f"{year}-tasa desocupación"
    ensure_directory_exists(folder_name)
    
    # Obtener las regiones y los meses únicos del DataFrame
    regiones = df['Región'].unique()
    meses = df['Mes'].unique()
    
    #colores asignados a cada sexo
    colores = {'Hombres': 'blue', 'Mujeres': 'orange'}
    
    # Determinar el límite máximo del eje y para todos los gráficos
    max_valor = df[['Hombres', 'Mujeres']].max().max()
    
    # Iterar sobre cada región para graficar
    for region in regiones:
        # Filtrar el DataFrame para la región actual
        df_region = df[df['Región'] == region]
        
        # Crear una nueva figura y un eje para el gráfico
        fig, ax = plt.subplots(figsize=(10, 6))
        
        # Iterar sobre los sexos ('Hombres' y 'Mujeres')
        for sexo in ['Hombres', 'Mujeres']:
            # Obtener los valores de desocupación para el sexo actual
            valores_sexo = df_region[sexo].tolist()
            
            # Graficar la línea
            ax.plot(meses, valores_sexo, label=sexo, color=colores[sexo], marker='o')
        
        # Configurar etiquetas y título del gráfico
        ax.set_xlabel('Mes')
        ax.set_ylabel('Tasa de Desocupación (expresado como porcentaje de la fuerza de trabajo)')
        
        # Añadir el subtítulo
        plt.suptitle(f'Tasa de Desocupación en {region} ({year})', fontsize=14, y=1)
        
        # Añadir el título principal
        ax.set_title('% de personas desocupadas respecto a fuerza de trabajo según INE (Instituto Nacional de Estadísticas)', fontsize=10, pad=20)
        
        # Configurar las etiquetas del eje x
        ax.set_xticks(meses)
        ax.set_xticklabels(meses, rotation=45, ha='right')
        
        # Configurar los límites del eje y
        ax.set_ylim(0, max_valor)
        
        # Añadir la leyenda
        ax.legend()
        
        # Ajustar el diseño
        plt.tight_layout()  # Ajusta el área de dibujo para dejar espacio al subtítulo
        
        # Guardar el gráfico en la carpeta especificada
        file_name = os.path.join(folder_name, f'Tasa de Desocupación en {region} año ({year}).png')
        plt.savefig(file_name)
        
        # Cerrar la figura para liberar memoria
        plt.close(fig)
    
    print(f"Todos los gráficos han sido guardados en la carpeta '{folder_name}'.")
    
    
def ensure_directory_exists(directory):
    """
    Verifica si una carpeta existe y, si no, la crea.
    
    Args:
    - directory (str): Ruta de la carpeta a verificar o crear.
    """
    if not os.path.exists(directory):
        os.makedirs(directory)

In [3]:
start_date = int(dataframe.head(1)["DTI_CL_TRIMESTRE_MOVIL"].str[0:4].iloc[0])
end_date = int(dataframe.tail(1)["DTI_CL_TRIMESTRE_MOVIL"].str[0:4].iloc[0])
end_date


try:
    year = '2024'
    converted_year = int(year)

    if(converted_year < start_date or converted_year > end_date):
        print(f'No se puede analizar el año ingresado.\nEl año debe estar entre {start_date} y {end_date}')

    else:
        df_prepared = prepare_data(dataframe_clean, year)
        # Graficar por región
        plot_by_region(df_prepared, year)

except Exception as e:
    print(e)
    print(f'Solo dígitos son compatibles en el formato YYYY. Ejemplo 2020')
     



Todos los gráficos han sido guardados en la carpeta '2024-tasa desocupación'.
